**Mount Drive & Install Packages**

First the directory containing the data must be mounted to the Colab virtual machine

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Import relevant libraries for use

In [ ]:
import tensorflow as tf
from tensorflow import keras

tf.__version__
keras.__version__

Paths for training data and test data

In [ ]:
training_data_path = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed-N4/train/'
valid_data_path = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed-N4/test/'

Produce trainin set generators to flow images thorugh from preprocessed directory

In [ ]:

trainGenerator = tf.keras.preprocessing.image.ImageDataGenerator()

validGenerator = tf.keras.preprocessing.image.ImageDataGenerator()

flowTrainGenerator = trainGenerator.flow_from_directory(training_data_path, target_size=(224,224), color_mode='rgb',shuffle=True,interpolation="nearest",batch_size=64, class_mode='categorical')

flowValidGenerator = validGenerator.flow_from_directory(valid_data_path, target_size=(224,224), color_mode='rgb',shuffle=True,interpolation="nearest",batch_size=64, class_mode='categorical')

**Initialise Metrics**

List of metrics ready for Keras


In [ ]:
METRICS = [
           tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None),
           tf.keras.metrics.Precision(thresholds=None, top_k=None, class_id=None, name=None, dtype=None),
           tf.keras.metrics.Recall(thresholds=None, top_k=None, class_id=None, name=None, dtype=None),
           tf.keras.metrics.TruePositives(thresholds=None, name=None, dtype=None),
           tf.keras.metrics.TrueNegatives(thresholds=None, name=None, dtype=None),
           tf.keras.metrics.FalsePositives(thresholds=None, name=None, dtype=None),
           tf.keras.metrics.FalseNegatives(thresholds=None, name=None, dtype=None)          
]

**Implementation of ResNet18**

My own implementation of ResNet50 in Keras with my own adaptations

In [ ]:
#Implementation of ResNet18 using the Keras Functional API which can handle non-linear topologies

#Input layer

inputShape = (224,224,3)

inputs = tf.keras.Input(shape=inputShape)

#Create identity building block and convolutional building blocks

def identityBlock(x, f, d):

    shortcut = x

    if x.shape[-1] != f:
      shortcut = tf.keras.layers.Conv2D(f,(1,1), padding='same')(x)

    x = tf.keras.layers.Conv2D(f,(d,d), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(f,(d,d),padding='same')(x) 
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.add([x, shortcut])

    x = tf.keras.activations.relu(x)

    return x

#Initial layers of module

x = tf.keras.layers.Conv2D(64,(7,7), input_shape=inputShape, strides = (2,2))(inputs)

x = tf.keras.layers.MaxPool2D(pool_size = (2), strides = (2,2))(x)

#Residual block section

residualLayer = identityBlock(x, 64, 3)

residualLayer = identityBlock(residualLayer, 128, 3)

residualLayer = identityBlock(residualLayer, 256, 3)

residualLayer = identityBlock(residualLayer, 512, 3)

#Final max pool section

#Penultimate average pooling layer

x = tf.keras.layers.GlobalAveragePooling2D()(residualLayer)

#Must flatten the matrix to obtain the ultimate classification

x = tf.keras.layers.Flatten()(x)

#Add the last fully connected layer to classify into 5 class types

output = tf.keras.layers.Dense(5, activation = "softmax")(x)

#Create the model 

resNetModel = tf.keras.Model(inputs, output)

#Formulate optimiser to give fine grain control over learning rate and momentum

SGDoptimiser = tf.keras.optimizers.SGD(momentum = 0.8, learning_rate=0.001)

RMSPropOptimiser = tf.keras.optimizers.RMSprop(learning_rate=0.01, momentum=0.9)

#compile the above model and specify the optimisation function for Stochastic Gradient Decent and Mean Square Error loss function

resNetModel.compile(optimizer=SGDoptimiser, loss='categorical_crossentropy', metrics=METRICS)

#Print message to indicate model compilation has been completed

print('Model compilation complete')

resNetModel.summary()

resNetModel.save('/content/drive/My Drive/AD Project/Model Saves/ResNet18-Multiclass')



**Visualisation of ResNet-18**

Graph of architecture is shown below

In [ ]:
#Displays the above model's architecture visually- saved as png in model saves folder

import matplotlib.pyplot as plt
import cv2 as cv

pathToModel = '/content/drive/My Drive/AD Project/Model Saves/ResNet18Multiclass.png'

tf.keras.utils.plot_model(resNetModel, show_shapes=True, show_layer_names=True, to_file=pathToModel)

modelImage = cv.imread('/content/drive/My Drive/AD Project/Model Saves/ResNet18Multiclass.png')

Fit model to dataset

In [ ]:
fittedModel = resNetModel.fit(flowTrainGenerator, epochs=500, steps_per_epoch=17, validation_data=flowValidGenerator, validation_steps=3, verbose=1)

**Visualisations of Performance**

In [ ]:
import matplotlib.pyplot as plt

print(fittedModel.history)

#Plot the accuracy change by number of epochs

plt.plot(fittedModel.history["categorical_accuracy"])
plt.ylabel("Categorical Accuracy")
plt.xlabel("Epoch")
plt.title("Categorical Accuracy by Epoch")
plt.show()

#Plot the change in loss by number of epochs

plt.plot(fittedModel.history["loss"])
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.title("Loss by Epoch")
plt.show()

#Plot the validation accuracy change by number of epochs

plt.plot(fittedModel.history["val_categorical_accuracy"])
plt.ylabel("Validation Accuracy")
plt.xlabel("Epoch")
plt.title("Validation Accuracy by Epoch")
plt.show()

#Plot the change in validaiton loss by number of epochs

plt.plot(fittedModel.history["val_loss"])
plt.ylabel("Validation Loss")
plt.xlabel("Epoch")
plt.title("Validation Loss by Epoch")
plt.show()

**Key Statistics**

Report of key metrics as part of report:

- Accuracy
- F-1 Score
- Precision
- Recall

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay

F1_Score = (2*(((fittedModel.history['val_precision_1'][-1])*(fittedModel.history['val_recall_1'][-1]))/((fittedModel.history['val_precision_1'][-1])+(fittedModel.history['val_recall_1'][-1]))))

print("Accuracy is:")

print(fittedModel.history['categorical_accuracy'][-1])

print("F-1 Score is:")

print(F1_Score)

print("Precision is:")

print(fittedModel.history['val_precision_1'][-1])

print("Recall is:")

print(fittedModel.history['val_recall_1'][-1])

print("Confusion Matrix")

classPrediction = resNetModel.predict(flowValidGenerator, verbose=1)

classPrediction = np.argmax(classPrediction, axis=1)

#Use skikit learn maxtrix generator to make confusion matrix with numpy array

confusionMatrix = confusion_matrix(flowValidGenerator.classes,classPrediction)

displayMatrix = ConfusionMatrixDisplay(confusionMatrix, display_labels=flowTrainGenerator.classes)

displayMatrix.plot()

In [ ]:
#Save model with weights

resNetModel.save('/content/drive/My Drive/AD Project/Multiclass CNNs/Model Saves/ResNet18-Multiclass.h5')